<h1 align=center><font size = 5> Build a Regression Model with Keras with Data Normalization</font></h1>

<h1 align=left><font size = 3>D. Increase the number of hidden layers</font></h1>


Use the Keras library to build a neural network with the following:
- Three hidden layer of 5 nodes, and a ReLU activation function
- Use the **adam** optimizer and the **mean squared error** as the loss function.


Let's start by importing the pandas and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

Let's download the data and read it into a <em>pandas</em> dataframe.



In [2]:
concrete_data = pd.read_csv(' https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


**Split data into predictors and target and randomly spliting the data into a training and test sets by holding 30% of the data for testing.**

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

X_train, X_test, y_train, y_test = train_test_split( predictors, target, test_size=0.3, random_state=30)

In [8]:
n_cols = predictors.shape[1] # number of predictors

To normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
X_train_norm = (X_train - X_train.mean()) / X_train.std()
X_train_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
590,-0.419281,-0.852944,-0.859625,0.207241,-1.077712,1.890821,0.202965,-0.623796
197,-0.839210,-0.852944,0.723351,-0.720695,0.194690,0.424266,1.675144,0.219283
184,-0.571191,-0.852944,0.663497,0.373439,-0.314271,-0.088507,1.225662,-0.692619
57,-0.801474,0.688746,-0.859625,0.498087,-1.077712,0.058932,0.660022,-0.692619
200,-0.877913,-0.852944,1.115551,-0.882276,0.245586,1.515049,0.388566,-0.503356


In [10]:
X_test_norm = (X_test - X_test.mean()) / X_test.std()
X_test_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
3,0.484070,0.768370,-0.816436,2.210307,-0.952639,-0.481133,-2.186062,4.273190
517,-0.731903,-0.737481,1.344010,1.143961,-0.676270,-0.356708,0.322187,-0.603113
107,0.402074,2.374992,-0.816436,0.067921,0.721832,-0.347999,-1.387542,-0.603113
659,-1.604981,0.996252,-0.816436,1.022785,-0.952639,-0.403990,0.903809,0.527426
381,0.321009,0.705388,-0.816436,-1.812725,0.006524,1.982477,-0.356373,-0.317073


We can now import keras and its classes Sequential and Dense

In [11]:
import keras

In [12]:
from keras.models import Sequential
from keras.layers import Dense

We can now build our neural network :

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    # Three hidden layer of 5 nodes, and a ReLU activation function
    model.add(Dense(5, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1))

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Train the model on the training data using 50 epochs while randomly spliting the data into a training and test sets by holding 30% of the data for testing.

In [14]:
# build the model
model = regression_model()

In [15]:
# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=2)

Epoch 1/50
23/23 - 1s - loss: 3510.8838 - val_loss: 3061.1230 - 1s/epoch - 61ms/step
Epoch 2/50
23/23 - 0s - loss: 2679.3101 - val_loss: 2557.7258 - 71ms/epoch - 3ms/step
Epoch 3/50
23/23 - 0s - loss: 2299.3430 - val_loss: 2274.0190 - 82ms/epoch - 4ms/step
Epoch 4/50
23/23 - 0s - loss: 2076.9028 - val_loss: 2092.9983 - 85ms/epoch - 4ms/step
Epoch 5/50
23/23 - 0s - loss: 1931.0115 - val_loss: 1972.2201 - 80ms/epoch - 3ms/step
Epoch 6/50
23/23 - 0s - loss: 1831.7710 - val_loss: 1886.5563 - 88ms/epoch - 4ms/step
Epoch 7/50
23/23 - 0s - loss: 1760.3593 - val_loss: 1825.5441 - 81ms/epoch - 4ms/step
Epoch 8/50
23/23 - 0s - loss: 1708.6885 - val_loss: 1778.0674 - 75ms/epoch - 3ms/step
Epoch 9/50
23/23 - 0s - loss: 1668.6326 - val_loss: 1741.5906 - 75ms/epoch - 3ms/step
Epoch 10/50
23/23 - 0s - loss: 1637.5298 - val_loss: 1712.6671 - 82ms/epoch - 4ms/step
Epoch 11/50
23/23 - 0s - loss: 1613.1658 - val_loss: 1689.1306 - 83ms/epoch - 4ms/step
Epoch 12/50
23/23 - 0s - loss: 1593.3445 - val_loss: 

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

In [16]:
from sklearn.metrics import mean_squared_error

# Initialize an empty list and a counter
MSE_list = []
counter = 1

# Fill the list with elements using a while loop
while counter <= 50:
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=2)
    y_pred = model.predict(X_test)
    MSE_list.append(mean_squared_error(y_test, y_pred))
    counter += 1

print(MSE_list)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
23/23 - 0s - loss: 145.0314 - val_loss: 156.7706 - 108ms/epoch - 5ms/step
Epoch 27/50
23/23 - 0s - loss: 143.5469 - val_loss: 156.1313 - 104ms/epoch - 5ms/step
Epoch 28/50
23/23 - 0s - loss: 142.7950 - val_loss: 155.6887 - 113ms/epoch - 5ms/step
Epoch 29/50
23/23 - 0s - loss: 140.8890 - val_loss: 155.1954 - 123ms/epoch - 5ms/step
Epoch 30/50
23/23 - 0s - loss: 139.6523 - val_loss: 154.8934 - 106ms/epoch - 5ms/step
Epoch 31/50
23/23 - 0s - loss: 138.3101 - val_loss: 154.4493 - 107ms/epoch - 5ms/step
Epoch 32/50
23/23 - 0s - loss: 137.6989 - val_loss: 154.1718 - 156ms/epoch - 7ms/step
Epoch 33/50
23/23 - 0s - loss: 136.2643 - val_loss: 153.7858 - 118ms/epoch - 5ms/step
Epoch 34/50
23/23 - 0s - loss: 135.4539 - val_loss: 153.3793 - 112ms/epoch - 5ms/step
Epoch 35/50
23/23 - 0s - loss: 133.7095 - val_loss: 152.8894 - 151ms/epoch - 7ms/step
Epoch 36/50
23/23 - 0s - loss: 132.9803 - val_loss: 152.6825 - 83ms/epoch 

In [18]:
import statistics
mean = statistics.mean(MSE_list)
standard_deviation = statistics.stdev(MSE_list)

print("Mean after Normalization with 3 layers of 50 nodes each :", mean)
print("Standard Deviation after Normalization with 3 layers of 50 nodes each :", standard_deviation)

Mean after Normalization with 3 layers of 50 nodes each : 58.64237308231651
Standard Deviation after Normalization with 3 layers of 50 nodes each : 18.90077769082064


**Part A:**

Mean: 61.02331363622501

Standard Deviation: 14.615134162091586

**Part B:**

Mean after Normalization: 58.20324839422981

Standard Deviation after Normalization: 10.494701801911273

**Part C:**

Mean after Normalization and with 100 epochs: 67.28571954024014

Standard Deviation after Normalization and with 100 epochs: 73.69759301609396

**Part D:**

Mean after Normalization with 3 layers of 50 nodes each : 58.64237308231651

Standard Deviation after Normalization with 3 layers of 50 nodes each : 18.90077769082064

**Conclusion**

- Normalizing the input data allowed us to have more precise results
- Increasing the number of epochs 50 --> 100 made us reach lower loss values but still shows inconsistent MSE values
- Increasing the number of epochs isn't always the solution for a better accuracy and model performance and it lead in this exemple in overftiing: The model seems to be memorising the data instead of learning it.
- By increasing the number of hidden layers 1 --> 3 we didn't reach better results than in part B. It resulted in the overfitting of the model.

